In [62]:
import sys
sys.path.append('../')

In [63]:
from scripts.utils import data_path
from scripts.cross_match_scripts import my_scaler_forward
import pandas as pd
from tqdm import tqdm
pd.set_option('display.max_columns', 500)
import seaborn as sns
import numpy as np
%matplotlib inline

# Catalogs loading and preprocessing

In [64]:
photo_cat = pd.read_pickle(data_path+'csc_allsky_desi_r30_gaia_dered_training.gz_pkl', compression='gzip')
x_ray_flux_lim = 3e-15
photo_cat = photo_cat.query('(csc_flux_05_2 > @x_ray_flux_lim) | is_counterpart == False')
photo_cat = photo_cat.replace([np.inf, -np.inf], np.nan)
photo_cat.sample(5)

,csc_name,csc_ra,csc_dec,csc_flux_05_2,csc_radec_err,csc_r_98,release,objid,brickid,ra,dec,brick_primary,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,snr_g,snr_r,snr_z,snr_w1,snr_w2,snr_w3,snr_w4,parallax,parallax_ivar,pmra,pmra_ivar,pmdec,pmdec_ivar,dist_arcsec,desi_id,all_mag_g,rel_mag_g,rel_dered_mag_g,all_mag_r,rel_mag_r,rel_dered_mag_r,all_mag_z,rel_mag_z,rel_dered_mag_z,all_mag_w1,rel_mag_w1,rel_dered_mag_w1,vega_mag_w1,all_mag_w2,rel_mag_w2,rel_dered_mag_w2,vega_mag_w2,all_mag_w3,rel_mag_w3,rel_dered_mag_w3,vega_mag_w3,all_mag_w4,rel_mag_w4,rel_dered_mag_w4,vega_mag_w4,rel_desi_flux_corr_z,rel_dered_g_r,rel_dered_g_z,rel_dered_r_z,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4,r_false_003_dens_10_30,num_srcs_sep_less_r_false,is_counterpart
532915,2CXO J053254.8-622939,83.228370,-62.494347,NaN,0.532429,NaN,9010,14695,37540,83.222859,-62.499013,1,0.338334,0.471834,0.315787,2.164459,2.835420,-17.838997,5089.49900,612.13150,510.365840,113.654480,7.139467,2.603002,0.005760,0.000063,23.465837,23.173529,23.672089,21.649559,21.361046,NaN,13.232716,8.370811,10.659335,3.366566,5.783388,4.574616,-1.353847,40.529064,0.0,0.0,0.0,0.0,0.0,0.0,19.132749,9010_37540_14695,23.676636,23.676636,23.465837,23.315527,23.315527,23.173529,23.751514,NaN,NaN,21.661627,21.661627,21.649559,18.962627,21.368457,21.368457,21.361046,18.029457,NaN,NaN,NaN,NaN,13.233312,13.233312,13.232716,6.613312,NaN,0.292308,NaN,NaN,NaN,1.812483,0.288513,NaN,9.940813,NaN,0.493633,1,False
182693,2CXO J013015.7+214830,22.565718,21.808387,NaN,1.432105,NaN,9010,4470,452927,22.560558,21.805298,1,0.180300,0.361037,1.233817,14.330616,14.284651,46.813957,211.78352,1287.71600,517.495000,66.301230,2.857235,0.648968,0.001360,0.000018,24.167898,23.476710,22.199486,19.598340,19.606071,18.32262,16.684725,6.470011,8.213057,10.046424,24.223555,11.507505,1.726538,0.908792,0.0,0.0,0.0,0.0,0.0,0.0,20.521927,9010_452927_4470,24.360011,24.360011,24.167898,23.606121,23.606121,23.476710,22.271873,22.271873,22.199486,19.609338,19.609338,19.598340,16.910338,19.612826,19.612826,19.606071,16.273826,18.324062,NaN,NaN,NaN,16.685270,NaN,NaN,NaN,2.201874e-15,0.691188,1.968412,1.277224,2.601146,3.870639,-0.007731,NaN,NaN,NaN,1.274555,1,False
1499108,2CXO J204746.9-062025,311.945499,-6.340288,NaN,0.441520,NaN,9010,7912,295660,311.946676,-6.343211,1,0.110968,0.464582,2.205027,-1.905861,-4.344537,-3.918520,-667.98206,1613.96680,542.812500,110.615616,2.854360,0.679381,0.000603,0.000008,24.702229,23.207876,21.571844,NaN,NaN,NaN,NaN,4.458054,10.823979,23.191137,-3.219924,-3.580966,-0.096241,-1.910760,0.0,0.0,0.0,0.0,0.0,0.0,11.334945,9010_295660_7912,24.887006,24.887006,24.702229,23.332344,23.332344,23.207876,21.641465,21.641465,21.571844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.925087e-15,1.494353,3.130385,1.636032,NaN,NaN,NaN,NaN,NaN,NaN,0.949997,1,False
728498,2CXO J095107.5+390557,147.781645,39.099401,NaN,6.716253,NaN,9011,2038,538986,147.786295,39.096971,1,0.340202,1.582540,5.568395,16.265642,7.569196,-0.109664,132.00084,500.20053,123.199104,66.165000,3.094496,0.773070,0.001235,0.000015,23.600471,21.954336,20.609230,19.467804,20.299908,NaN,17.198360,7.608676,17.565414,45.294365,28.613184,6.655168,-0.003853,0.517937,0.0,0.0,0.0,0.0,0.0,0.0,15.662119,9011_538986_2038,23.670658,23.670658,23.600471,22.001613,22.001613,21.954336,20.635675,20.635675,20.609230,19.471822,19.471822,19.467804,16.772822,20.302376,20.302376,20.299908,16.963376,NaN,NaN,NaN,NaN,17.198558,NaN,NaN,NaN,9.525657e-15,1.646135,2.991241,1.345106,1.141426,1.654428,-0.832104,NaN,NaN,NaN,2.015249,1,False
1180729,2CXO J135615.3+182323,209.063817,18.389945,NaN,0.689155,NaN,9010,2429,436020,209.060203,18.384998,1,2.797659,4.152776,4.335479,4.191943,-1.649938,12.151027,-499.46036,257.79904,65.284

In [65]:
print('fraction of missed measurements per is_counterpart per filter, in per cent:')
photo_cat.groupby('is_counterpart')[['rel_mag_g',	'rel_mag_r',	'rel_mag_z',	'rel_mag_w1',	'rel_mag_w2',	'rel_mag_w3',	'rel_mag_w4']].agg(lambda x: np.round(100*np.mean(np.
isnan(x)),1))

fraction of missed measurements per is_counterpart per filter, in per cent:


,rel_mag_g,rel_mag_r,rel_mag_z,rel_mag_w1,rel_mag_w2,rel_mag_w3,rel_mag_w4
is_counterpart,,,,,,,
False,21.1,14.0,24.1,59.3,76.1,95.6,85.9
True,7.4,4.5,2.0,5.0,9.2,70.8,79.5


In [66]:
features_cols = ['rel_dered_mag_g','rel_dered_mag_r','rel_dered_mag_z'] 
features_cols += ['rel_dered_g_r', 'rel_dered_r_z', 'rel_dered_g_z']


features_cols += ['rel_dered_mag_w1', 'rel_dered_mag_w2']
features_cols += ['rel_dered_z_w1', 'rel_dered_r_w2', 'rel_dered_w1_w2']

features_cols += ['rel_dered_mag_w3', 'rel_dered_mag_w4']
features_cols += ['rel_dered_z_w3', 'rel_dered_r_w4', 'rel_dered_w3_w4']


target_col = ['is_counterpart']

photo_cat = photo_cat[features_cols+target_col+['csc_flux_05_2']]

#photo_cat= photo_cat.dropna(subset = features_cols, how = 'any')

print(photo_cat.is_counterpart.value_counts())
photo_cat = photo_cat.sample(frac=1)
photo_cat.reset_index(inplace=True, drop = True)

photo_cat_scaled = my_scaler_forward(photo_cat)

photo_cat_scaled.to_pickle('./model/train_test_validation_data.pkl')

photo_cat_scaled.head(15)


False    372584
True      36277
Name: is_counterpart, dtype: int64
rel_dered_mag_g scaled by 1/35
rel_dered_mag_r scaled by 1/35
rel_dered_mag_z scaled by 1/35
rel_dered_g_r scaled by 1/10
rel_dered_r_z scaled by 1/10
rel_dered_g_z scaled by 1/10
rel_dered_mag_w1 scaled by 1/35
rel_dered_mag_w2 scaled by 1/35
rel_dered_z_w1 scaled by 1/10
rel_dered_r_w2 scaled by 1/10
rel_dered_w1_w2 scaled by 1/10
rel_dered_mag_w3 scaled by 1/35
rel_dered_mag_w4 scaled by 1/35
rel_dered_z_w3 scaled by 1/10
rel_dered_r_w4 scaled by 1/10
rel_dered_w3_w4 scaled by 1/10


,rel_dered_mag_g,rel_dered_mag_r,rel_dered_mag_z,rel_dered_g_r,rel_dered_r_z,rel_dered_g_z,rel_dered_mag_w1,rel_dered_mag_w2,rel_dered_z_w1,rel_dered_r_w2,rel_dered_w1_w2,rel_dered_mag_w3,rel_dered_mag_w4,rel_dered_z_w3,rel_dered_r_w4,rel_dered_w3_w4,is_counterpart,csc_flux_05_2
0,0.712913,0.700861,0.657693,0.042180,0.151091,0.193271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
1,0.567842,0.546925,0.535685,0.073210,0.039338,0.112548,0.564660,NaN,-0.101411,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
2,0.661302,0.652634,0.640308,0.030336,0.043143,0.073478,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
3,0.682663,0.657936,NaN,0.086546,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
4,0.652311,0.644779,0.637517,0.026359,0.025418,0.051777,0.620442,NaN,0.059761,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
5,0.500176,0.465735,0.447589,0.120544,0.063512,0.184055,0.467928,0.487020,-0.071187,-0.074497,-0.066822,NaN,NaN,NaN,NaN,NaN,False,NaN
6,NaN,0.701301,0.644745,NaN,0.197945,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
7,0.688056,0.641362,0.602759,0.163429,0.135112,0.298541,0.526830,0.512248,0.265752,0.451901,0.051036,0.47292,NaN,0.454436,NaN,NaN,True,7.942347e-15
8,0.668253,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.428455,NaN,NaN,NaN,False,NaN
9,0.658465,0.637415,0.631423,0.073672,0.020973,0.094646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
